<a href="https://colab.research.google.com/github/TK-Problem/Python-mokymai/blob/master/Scripts/nordstreet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
#@title Importuoti paketus
# for cleaning outputs
from IPython.display import clear_output

# for getting raw HTML
import requests

# for processing HTML
from bs4 import BeautifulSoup

# for data manipulation
import pandas as pd
import numpy as np

# interactive charts
from plotly.subplots import make_subplots
import plotly.graph_objects as go
import plotly.express as px

# specify plotly theme
import plotly.io as pio
pio.templates.default = "seaborn"

# clean console
clear_output()

## Atsiųsti duomenis

Duomenis yra atsisiunčiami iš angliškos nordstreed svetainės versijos (html kodas lengviau skaitomas). Mes naudomame `pandas` bibliotekos standardtinę funciją `read_html()`, kuri iš nurodyto puslapio suranda visas lenteles su `<table>` tag'u.

Paskutinį kartą duomenys nuskaityti **2023-07-02**.

In [2]:
%%time
# get link to detailed data
url = 'https://nordstreet.com/projects-summary'

# load all tables
tables = pd.read_html(url)

CPU times: user 574 ms, sys: 11.5 ms, total: 585 ms
Wall time: 1.45 s


In [3]:
#@title Finansavimą renkantys projektai
# currently available projects, i.e. projects waiting for investment
df_open = tables[0]

# remove columns unnececary columns
df_open = df_open[df_open.columns[1:-1]].copy()

# convert risk rating to letters
df_open['Project risk'] = df_open['Project risk'].apply(lambda x: x[0])

# convert interest rates to float
df_open['Project interest'] = df_open['Project interest'].apply(lambda x: float(x.split(" ")[0]))

# return first line
df_open.head()


,Project title,Goal,Fund types,Project LTV,Project interest,Project risk,Investors count,Project start,Intended project end
0,Business loan #00259,25 000 €,Other pledge,0,12.0,B,114,2023-07-07,2024-07-07


In [4]:
#@title Vykdomi projektai
# ongoing projects
df_ongoing = tables[1]

# remove columns unnececary columns
df_ongoing = df_ongoing[df_ongoing.columns[1:]].copy()

# convert goal eur to float
df_ongoing['Goal'] = df_ongoing['Goal'].apply(lambda x: float(x[:-2].replace(" ", "")))

# convert interest rates and annual returns to float
df_ongoing['Project interest'] = df_ongoing['Project interest'].apply(lambda x: float(x.split(" ")[0]))
df_ongoing['Annual return'] = df_ongoing['Annual return'].apply(lambda x: float(x.split(" ")[0]))

# convert risk rating to letters
df_ongoing['Project risk'] = df_ongoing['Project risk'].apply(lambda x: x[0])

df_ongoing.head()

,Project title,Goal,Fund types,Project LTV,Project interest,Annual return,Project risk,Investors count,Project start,Intended project end,Investors earnings,Payment of interest delay,Loan return delay
0,Business loan #00028,30000.0,Loan with another pledge,0,11.0,10.89,A,74,2021-01-13,2024-01-13,5114.38,-,-
1,"Legemyrane road 4, Moltustranda city, Norway (...",200000.0,First order mortgage,50,11.0,11.09,C,333,2021-02-04,2023-08-04,53586.40,-,-
2,Business loan #00042,30000.0,Loan with another pledge,0,11.0,10.80,A,103,2021-02-01,2024-02-01,5045.67,-,-
3,Business loan #00045,25000.0,Loan with another pledge,0,12.0,11.92,B,60,2021-01-29,2024-01-29,4676.95,-,-
4,"Legemyrane road 4, Moltustranda city, Norway (...",150000.0,First order mortgage,50,11.0,11.09,C,285,2021-02-13,2023-08-04,40043.78,-,-


In [5]:
#@title Išieškomos paskolos
# Recoverable loans
df_recoverable = tables[2]

# remove columns unnececary columns
df_recoverable = df_recoverable[df_recoverable.columns[1:]].copy()

# convert goal eur to float
df_recoverable['Goal'] = df_recoverable['Goal'].apply(lambda x: float(x[:-2].replace(" ", "")))

# convert interest rates and annual returns to float
df_recoverable['Project interest'] = df_recoverable['Project interest'].apply(lambda x: float(x.split(" ")[0]))
df_recoverable['Annual return'] = df_recoverable['Annual return'].apply(lambda x: float(x.split(" ")[0]))

# convert risk rating to letters
df_recoverable['Project risk'] = df_recoverable['Project risk'].apply(lambda x: x[0])

df_recoverable.head()

,Project title,Goal,Fund types,Project LTV,Project interest,Annual return,Project risk,Investors count,Project start,Intended project end,Interest paid,Payment of interest delay,Loan return delay
0,"Vilties str 20, Nemeikščiai village, Utenos co...",94000.0,First order mortgage,49,10.5,10.89,B,216,2020-10-18,2021-09-28,2.90,-,641 d
1,Business loan #00002,7500.0,Loan with another pledge,0,12.0,12.00,B,1,2020-10-22,2021-10-22,2 424.59,-,617 d
2,Business loan #00012,8000.0,Loan with another pledge,0,13.0,13.00,C,2,2020-12-02,2023-08-12,2.62,-,-
3,"Vilniaus str. 38B, Putiniškės, Vilnius distric...",190000.0,First order mortgage,56,11.5,12.07,B,240,2020-12-04,2021-12-04,0.00,-,574 d
4,Business loan #00013,35000.0,Loan with another pledge,0,12.0,11.91,B,30,2020-12-04,2021-12-04,0.00,-,574 d


In [6]:
#@title Išieškotos paskolos
# recovered loans
df_recovered = tables[3]

# remove columns unnececary columns
df_recovered = df_recovered[df_recovered.columns[1:]].copy()

# convert goal eur to float
df_recovered['Goal'] = df_recovered['Goal'].apply(lambda x: float(x[:-2].replace(" ", "")))

# convert interest rates and annual returns to float
df_recovered['Project interest'] = df_recovered['Project interest'].apply(lambda x: float(x.split(" ")[0]))
df_recovered['Annual return'] = df_recovered['Annual return'].apply(lambda x: float(x.split(" ")[0]))
df_recovered['Actual annual return'] = df_recovered['Actual annual return'].apply(lambda x: float(x.split(" ")[0]))

# convert risk rating to letters
df_recovered['Project risk'] = df_recovered['Project risk'].apply(lambda x: x[0])

df_recovered.head()

,Project title,Goal,Fund types,Project LTV,Project interest,Annual return,Project risk,Investors count,Project start,Intended project end,Actual end of project,Project duration,Investors earnings,Actual annual return
0,Business loan #00006,25000.0,Loan with another pledge,0,12.0,11.49,B,37,2020-10-20,2021-10-20,2021-12-03,409 d.,6 263.24 €,22.41
1,Business loan #00036,30000.0,Loan with another pledge,0,11.0,10.93,B,80,2021-01-27,2023-01-27,2021-10-13,259 d.,4 725.70 €,22.28
2,"Migūčionių str.2A, Elektrėnų municipality, Lit...",80000.0,Second order mortgage,54,12.5,12.85,B,114,2021-02-26,2022-02-26,2021-10-13,229 d.,13 975.95 €,27.96
3,Business loan #00088,50000.0,Loan with another pledge,0,11.0,11.95,A,14,2021-03-23,2022-03-23,2023-06-17,816 d.,7 790.51 €,13.07
4,Business loan #00127,35000.0,Loan with another pledge,0,11.5,11.92,B,34,2021-05-21,2021-11-21,2021-10-13,145 d.,5 734.47 €,41.52


In [7]:
#@title Užbaigti projektai
# finished projects
df_finished = tables[4]

# remove columns unnececary columns
df_finished = df_finished[df_finished.columns[1:]].copy()

# convert goal eur to float
df_finished['Goal'] = df_finished['Goal'].apply(lambda x: float(x[:-2].replace(" ", "")))

# convert interest rates and annual returns to float
df_finished['Project interest'] = df_finished['Project interest'].apply(lambda x: float(x.split(" ")[0]))
df_finished['Annual return'] = df_finished['Annual return'].apply(lambda x: float(x.split(" ")[0]))
df_finished['Actual annual return'] = df_finished['Actual annual return'].apply(lambda x: float(x.split(" ")[0]))

# convert risk rating to letters
df_finished['Project risk'] = df_finished['Project risk'].apply(lambda x: x[0])

df_finished.head()

,Project title,Goal,Fund types,Project LTV,Project interest,Annual return,Project risk,Investors count,Project start,Intended project end,Actual end of project,Project duration,Investors earnings,Actual annual return
0,"Vilnius, Kareivių str. 2B-401",6170.0,First order mortgage,69,13.0,13.0,A,14,2018-03-27,2020-03-27,2018-11-09,227 d.,499.64 €,13.020841
1,Šopen Development Loan with NT Deposit,61600.0,First order mortgage,0,12.0,12.0,A,16,2018-03-12,2019-03-12,2019-02-12,337 d.,6 942.30 €,12.206345
2,"Goštauto str. 8, Vilnius, Lithuania",9730.0,Second order mortgage,0,16.0,16.0,B,20,2018-04-20,2019-04-20,2019-04-20,365 d.,1 577.26 €,16.210277
3,"Naugarduko str. 26, Vilnius, Lithuania",60000.0,First order mortgage,52,10.0,10.0,A,13,2018-05-28,2019-05-28,2018-10-28,153 d.,616.71 €,10.000000
4,"Mėtų str. 114C, Lindiniškių km, Avižienių sen....",20000.0,First order mortgage,40,12.0,12.0,A,38,2018-08-31,2020-08-31,2018-09-30,30 d.,281.72 €,17.137967


In [8]:
#@title Atšaukti projektai
# canceled projects
df_cancel = tables[5]

# remove columns unnececary columns
df_cancel = df_cancel[df_cancel.columns[1:]].copy()

# convert goal eur to float
df_cancel['Goal'] = df_cancel['Goal'].apply(lambda x: float(x[:-2].replace(" ", "")))

# convert interest rates and annual returns to float
df_cancel['Project interest'] = df_cancel['Project interest'].apply(lambda x: float(x.split(" ")[0]))

# convert risk rating to letters
df_cancel['Project risk'] = df_cancel['Project risk'].apply(lambda x: x[0])

df_cancel.head()

,Project title,Goal,Fund types,Project LTV,Project interest,Project risk,Project start
0,"Birželio str. 23, Dainava, Kaunas",230000.0,First order mortgage,70,9.0,B,2018-12-09
1,"Stiklių g.7, Old town, Vilnius",210000.0,First order mortgage,70,10.0,B,2019-01-28
2,"A. Jakšto str. 6A, Naujamiestis eldership, Vil...",220000.0,First order mortgage,58,11.0,C,2019-05-16
3,"Sandėlių str. 44, Paneriai eldership, Vilnius",600000.0,First order mortgage,41,11.0,B,2019-07-31
4,"Eglių str. 35, Naujamiestis eldership, Vilnius...",53170.0,First order mortgage,51,11.0,B,2019-11-15


## Užbaigtų projektų analizė

Tikslas išnagrinėti visus tiek sėkmingai tiek nesėkmingai uždarytus/užbaigtus investicinius projektus ir įvertinti. Projektų tipai pagal baigtį:

* **Finished** - užbaigti
* **Recovered** - išieškoti projektai
* **Recoverable (new)** išieškomi projektai (pradėti išieškoti neatėjus numatytai pabaigimo terminui)
* **Recoverable (old)** išieškomi projektai (planuota pabaigimo data jau praėjusi)

In [9]:
#@title Filtruoti duomenis

# column names to select
cols = ['Project title', 'Goal', 'Fund types', 'Project LTV',
        'Project interest', 'Intended project end', 'Project risk']

# select all projects wich were intented to be finished by 1st of September
cond = pd.to_datetime(df_ongoing['Intended project end']) <= pd.Timestamp.now()

# filter out data
df = df_ongoing.loc[cond, cols]
df['Type'] = 'Ongoing'

# temp. DataFrame to select loans which are being recoverable
_df = df_recoverable.loc[:, cols]

# covert project intended end time to datetime
_df['Intended project end'] = pd.to_datetime(_df['Intended project end'])

# check whatever intended project end is before or after observation end
_df.loc[_df['Intended project end'] <= pd.Timestamp.now(), 'Type'] = 'Recoverable (old)'
_df.loc[_df['Intended project end'] > pd.Timestamp.now(), 'Type'] = 'Recoverable (new)'

# join tables
df = pd.concat([df, _df]).reset_index(drop = True)

# select recovered loans
_df = df_recovered.loc[:, cols]
_df['Type'] = 'Recovered'

# join tables
df = pd.concat([df, _df]).reset_index(drop = True)

# select finished loans
_df = df_finished.loc[:, cols]
_df['Type'] = 'Finished'

# join tables
df = pd.concat([df, _df]).reset_index(drop = True)
df.head()

,Project title,Goal,Fund types,Project LTV,Project interest,Intended project end,Project risk,Type
0,Business loan #00086,40000.0,Loan with another pledge,0,11.5,2023-04-13,B,Ongoing
1,Business loan #00117,50000.0,Loan with another pledge,0,11.0,2023-05-26,B,Ongoing
2,"Vilties str 20, Nemeikščiai village, Utenos co...",94000.0,First order mortgage,49,10.5,2021-09-28 00:00:00,B,Recoverable (old)
3,Business loan #00002,7500.0,Loan with another pledge,0,12.0,2021-10-22 00:00:00,B,Recoverable (old)
4,Business loan #00012,8000.0,Loan with another pledge,0,13.0,2023-08-12 00:00:00,C,Recoverable (new)


In [10]:
#@title Projekto baigčių pasiskirstymas
# get data for pie chart
df_pie = df.groupby(['Project risk', 'Type'])['Project title'].count().reset_index()

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=4,
                    specs=[[{'type':'domain'}, {'type':'domain'},
                            {'type':'domain'}, {'type':'domain'}]])

# risk rating A
fig.add_trace(go.Pie(labels=df_pie.loc[df_pie['Project risk'] == 'A', 'Type'],
                     values=df_pie.loc[df_pie['Project risk'] == 'A', 'Project title'],
                     name="Risk Rating A"),
              1, 1)

# risk rating B
fig.add_trace(go.Pie(labels=df_pie.loc[df_pie['Project risk'] == 'B', 'Type'],
                     values=df_pie.loc[df_pie['Project risk'] == 'B', 'Project title'],
                     name="Risk Rating B"),
              1, 2)

# risk rating C
fig.add_trace(go.Pie(labels=df_pie.loc[df_pie['Project risk'] == 'C', 'Type'],
                     values=df_pie.loc[df_pie['Project risk'] == 'C', 'Project title'],
                     name="Risk Rating C"),
              1, 3)

# risk rating D
fig.add_trace(go.Pie(labels=df_pie.loc[df_pie['Project risk'] == 'D', 'Type'],
                     values=df_pie.loc[df_pie['Project risk'] == 'D', 'Project title'],
                     name="Risk Rating D"),
              1, 4)

# Use `hole` to create a donut-like pie chart
fig.update_traces(hole=.4, hoverinfo="label+value+percent+name")

fig.update_layout(
    title_text="Outcome distributions by Risk Ratings",
    # Add annotations in the center of the donut pies.
    annotations=[dict(text='A', x=0.095, y=0.5, font_size=24, showarrow=False),
                 dict(text='B', x=0.37, y=0.5, font_size=24, showarrow=False),
                 dict(text='C', x=0.63, y=0.5, font_size=24, showarrow=False),
                 dict(text='D', x=0.905, y=0.5, font_size=24, showarrow=False)])

fig.show()

## Projekto baigčių priklausomybė pagal LTV

Bandome įvertinti paskolos santykio su įkeičiamu turtu (LTV - loan to value) įtaką projekto baigčiai.

In [11]:
#@title LTV skritulinės diagramos
# add categorical variable, LTV > 0
df.loc[df['Project LTV'] > 50, 'LTV'] = ">50% LTV"
df.loc[df['Project LTV'] <= 50, 'LTV'] = "<=50% LTV"
df.loc[df['Project LTV'] == 0, 'LTV'] = "0% LTV"

# get data for Sankey Diagram
df_sd = df.groupby(['LTV', 'Type'])['Project title'].count().reset_index()

# Create subplots: use 'domain' type for Pie subplot
fig = make_subplots(rows=1, cols=3,
                    subplot_titles=("LTV 0%", "LTV <=50%", "LTV >50%"),
                    specs=[[{'type':'domain'}, {'type':'domain'}, {'type':'domain'}]])

# 0% LTV
fig.add_trace(go.Pie(labels=df_sd.loc[df_sd['LTV'] == '0% LTV', 'Type'],
                     values=df_sd.loc[df_sd['LTV'] == '0% LTV', 'Project title'],
                     name="LTV 0%"),
              1, 1)

# LTV >50%
fig.add_trace(go.Pie(labels=df_sd.loc[df_sd['LTV'] == '<=50% LTV', 'Type'],
                     values=df_sd.loc[df_sd['LTV'] == '<=50% LTV', 'Project title'],
                     name="LTV <=50%"),
              1, 2)

# LTV >50%
fig.add_trace(go.Pie(labels=df_sd.loc[df_sd['LTV'] == '>50% LTV', 'Type'],
                     values=df_sd.loc[df_sd['LTV'] == '>50% LTV', 'Project title'],
                     name="LTV >50%"),
              1, 3)

# update layout
fig.update_layout(title_text="Outcome distributions by LTV")

fig.show()

In [12]:
#@title Srauto grafiko funkcija
def visualize_risk_rating(df, r_rating):
  # select data
  _df = df.loc[df['Project risk'] == r_rating].copy()

  # create dimensions
  risk_dim = go.parcats.Dimension(values=_df['Project risk'],
                                  categoryorder='category ascending',
                                  label="Risk Rating")
  ltv_dim = go.parcats.Dimension(values=_df['LTV'], label="LTV")
  out_dim = go.parcats.Dimension(values=_df['Type'], label="Loan outcome")

  fig = go.Figure(data = [go.Parcats(dimensions=[risk_dim, ltv_dim, out_dim],
                                     hoverinfo='count+probability',
                                     labelfont={'size': 18, 'family': 'Times'},
                                     tickfont={'size': 16, 'family': 'Times'})])

  # return figure to console
  fig.show()

In [13]:
#@title Atvaizduoti rizikos klasės srautą
# vizualize one risk rating results
RISK_CLASS = 'A' #@param ["A", "B", "C", "D"] {allow-input: false}
visualize_risk_rating(df, RISK_CLASS)

In [14]:
#@title Visų rizikos klasių srautai

# Create dimensions
risk_dim = go.parcats.Dimension(
    values=df['Project risk'],
    categoryorder='category ascending', label="Risk Rating"
)

ltv_dim = go.parcats.Dimension(values=df['LTV'], label="LTV")

out_dim = go.parcats.Dimension(values=df['Type'], label="Loan outcome")

fig = go.Figure(data = [go.Parcats(dimensions=[risk_dim, ltv_dim, out_dim],
                                   hoverinfo='count+probability',
                                   labelfont={'size': 18, 'family': 'Times'},
                                   tickfont={'size': 16, 'family': 'Times'})])


fig.show()